In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Purpose

I am going to go through the [task submissions](https://www.kaggle.com/mdabbert/ultimate-ufc-dataset/tasks?taskId=1285) for UFC 251 and see what set of predictions was the most profitable (if any!)

# Submissions

There are 3 submissions.

1. The dummy file (gives each fighter a .5 probability of winning.  Therefore will bet every underdog with greater than +100 odds.)

2. shortlikeafox's (my!) submission.  A GaussianNB model with 18 features chosen through a systematic process of feature set testing.

3. yeahhhhhboi's submission.  Here is the description from his dataset: "Still working on a regression to obtain a p value including various other datasets, these choices are derrived from comparing ratios of TKO/KO win/loss and submission win/loss against their opponents (from sherdog), comparing the 540 metric from fightmatrix.com, and weighing UFC wins larger than other MMA bouts included in their win loss record. Still very crude but I want to see how things pan out tomorrow. I cannot wait to see this historic fight card tomorrow, good luck on your bets!"

# Open All Submissions and add to a list

In [ ]:
#This will help us keep track of the submissions
sub_name_list = ['dummy', 'shortlikeafox', 'yeahhhhhboi']
score_list = [] #We can keep the scores here

#Put the submissions in dataframe form and add to a list.
sub_list = []
temp_df = pd.read_csv("/kaggle/input/ufc251dummysubmission/ufc-251-dummy.csv")
sub_list.append(temp_df)
temp_df = pd.read_csv("/kaggle/input/ufc-251-actual-submission/ufc-251-actual-submission.csv")
sub_list.append(temp_df)
temp_df = pd.read_csv("/kaggle/input/ufc-251-submission/ufc-251-submission.csv")
sub_list.append(temp_df)

# Open the csv that contains fight results and odds

In [ ]:
results_df = pd.read_csv("/kaggle/input/ultimate-ufc-dataset/most-recent-event.csv")

#We only need the fighter names, odds, and winner

results_df = results_df[['R_fighter', 'B_fighter', 'R_ev', 'B_ev', 'Winner']]
display(results_df)

# Iterate the submissions and see how everyone did!

In [ ]:
#Returns a specific bet EV based on winning_ev and probability.
def get_bet_ev(ev, prob):
    
    return(ev*prob - (1-prob)*100)

In [ ]:
#Used to determine the bet of each fight.  We will use probabilities and the ev to 
#determine profitable bets
def get_bet(R_prob, B_prob, R_ev, B_ev):
    red_ev = get_bet_ev(R_ev, R_prob)
    blue_ev = get_bet_ev(B_ev, B_prob)
    if red_ev > 0:
        return('Red')
    if blue_ev > 0:
        return('Blue')
    
    return 'None'

In [ ]:
def get_profit(winner, bet, R_ev, B_ev):
    if bet == 'None':
        return 0
    if (bet == 'Blue' and winner == 'Blue'):
        return B_ev
    if (bet == 'Red' and winner == 'Red'):
        return R_ev
    else:
        return (-100)

In [ ]:
#Let's make a helper function to make this easier

def get_score(sub, results):
#    display(sub)
#    display(results)
    #Let's merge the two dataframes
    merge_df = pd.merge(sub, results)
    #display(merge_df)
    #We can get the proper bet by using a lambda function
    merge_df['Bet'] = merge_df.apply(lambda x: get_bet(x['R_prob'],x['B_prob'],x['R_ev'],x['B_ev']), axis=1)
    merge_df['Profit'] = merge_df.apply(lambda x: get_profit(x['Winner'], x['Bet'], x['R_ev'], x['B_ev']), axis=1)
    display(merge_df)
    return(sum(merge_df['Profit']))

# Submission \#1: Dummy Submission

These are the results for the dummy submission.

In [ ]:
z = 0
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

Dummy only bets on underdogs that have odds greater than +100.  This was not a good night for the underdogs.  This submission won 2 out of 12 bets and abstained from one fight.  The total return for this model was -741 units.

# Submission \#2: shortlikeafox's Submission

In [ ]:
z = 1
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

shortlikeafox won 8 bets out of 12 for a profit of 36.34 units.  The late added fight of Tybura / Grishin was never updated to this dataset so that's why there is a fight missing from the dataframe.

# Submission \#3: yeahhhhhboi's Submission 

In [ ]:
z = 2
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

yeahhhhhboi saw a profit of 9.12 units on 8 winning bets out of 13 fights.

# Final Results

* dummy: -741 units
* shortlikeafox: +36.34 units
* yeahhhhhboi: + 9.12 units

shortlikeafox is the winner!